In [1]:
#from nltk.tag import pos_tag
#from nltk.tokenize import word_tokenize
#from nltk.grammar import DependencyGrammar
#from nltk.parse import (
#    DependencyGraph,
#    ProjectiveDependencyParser,
#    NonprojectiveDependencyParser,
#    )
#from nltk.parse.stanford import StanfordParser
#from nltk.parse.corenlp import CoreNLPParser
import pandas as pd
import spacy


In [2]:
parser = spacy.load("en_core_web_lg")

In [3]:
data = pd.read_csv('Datasets/Fig-QA Dataset for PCA Copy.csv')

In [5]:

# Use spacy parser to extract subject of each paraphrase; lookup concreteness value in concreteness file; save new subj and concreteness columns
conc_ratings = pd.read_excel('Datasets/concreteness ratings.xlsx')
subj_list = []
subj_lem_list = []
conc_list = []
for sent in data['ending']:
    subj = ""
    subj_lem = ""
    conc = None
    parsed = parser(sent)
    for token in parsed:
        if token.dep_ in ('nsubj','nsubjpass','csubj'):
            subj = token.text.lower()
            subj_lem = token.lemma_.lower()
            try:
                conc = conc_ratings[conc_ratings['Word_lower']==subj]['Conc.M'].values[0]
            except IndexError:
                # if word not found, try lemmatising
                try:
                    conc = conc_ratings[conc_ratings['Word_lower']==subj_lem]['Conc.M'].values[0]
                except IndexError:
                    conc = None
            break
    subj_list.append(subj)
    subj_lem_list.append(subj_lem)
    conc_list.append(conc)
    
data['subj'] = subj_list
data['subj lem'] = subj_lem_list
data['concreteness'] = conc_list

data.to_csv('Datasets/Fig-QA parsed.csv', index=False)

In [6]:

# Use spacy parser to extract root and tense of each paraphrase; save new root and tense columns
data = pd.read_csv('Datasets/Fig-QA parsed.csv')

conc_ratings = pd.read_excel('Datasets/concreteness ratings.xlsx')
root_list = []
tense_list = []
for sent in data['ending']:
    conc = None
    parsed = parser(sent)
    for token in parsed:
        if token.dep_ == "ROOT":
            root = token.text.lower()
            try:
                tense = token.morph.get("Tense")[0]
            except IndexError:
                    # if infinitive, get auxiliary verb
                    for token in parsed:
                        if token.dep_ == "aux":
                            try:
                                token.morph.get("Tense")[0]
                            except IndexError:
                                tense = None
                            break
            break
            
    root_list.append(root)
    tense_list.append(tense)
    
data['root'] = root_list
data['tense'] = tense_list

data.to_csv("Datasets/Fig-QA parsed.csv", index=False)